In [ ]:
from pypowsybl_jupyter import display_nad
import ipywidgets as widgets
import pypowsybl as pp

In [ ]:
network = pp.network.create_four_substations_node_breaker_network()

In [ ]:
#declare some example callbacks, to simply log the events in a output box
out1 = widgets.Output(layout=widgets.Layout(width='300px', height="600px", overflow="auto"))
def print_infos(data):   
    with out1:
        print(data)
print_infos('Events log:')

def test_select_node_callback(event):
        print_infos('Selected node: ' + str(event.selected_node))

def test_select_menu_callback(event):
        print_infos('Selected menu: ' + str(event.selected_menu))

def test_move_node_callback(event):
        print_infos('Moved node: ' + str(event.moved_node))

def test_move_text_node_callback(event):
        print_infos('Moved text node: ' + str(event.moved_text_node))

In [ ]:
#demonstrates how-to create on-hover info on supported equipments, in the NAD
def format_to_html_table(row, id, type):
    table = (
        row.to_frame()
        .style.set_caption(f"{type}: {id}")
        .set_table_styles(
            [
                {
                    "selector": "caption",
                    "props": "caption-side: top; font-weight: bold; background-color: #f8f8f8; border-bottom: 1px solid #ddd; width: fit-content; white-space: nowrap;",
                },
                {
                    "selector": "th",
                    "props": "text-align: left; font-weight: bold; background-color: #f8f8f8;",
                },
                {
                    "selector": "td",
                    "props": "text-align: left;",
                },
            ]
        )
        .format(precision=3, thousands=".", decimal=",")
        .set_table_attributes('border="0"')
        .hide(axis="columns")
        .to_html()
    )
    return table

def get_equipment_info(id, type):
    if type == 'LINE':
        return format_to_html_table(network.get_lines().loc[id], id, type)
    elif type == 'HVDC_LINE':
        return format_to_html_table(network.get_hvdc_lines().loc[id], id, type)
    elif type in [ 'PHASE_SHIFT_TRANSFORMER', 'TWO_WINDINGS_TRANSFORMER']:
        return format_to_html_table(network.get_2_windings_transformers().loc[id], id, type)
    elif type == 'THREE_WINDINGS_TRANSFORMER':
        return format_to_html_table(network.get_3_windings_transformers().loc[id], id, type)        
    return f"Equipment of type '{type}' with id '{id}'"

In [ ]:
#create a NAD widget for a network and activate the callbacks
nad_widget=display_nad(network.get_network_area_diagram(depth=4), enable_callbacks=True, popup_menu_items=['Item1', 'Item2', 'Item3'], 
                       on_hover_func=get_equipment_info)
nad_widget.on_select_node(test_select_node_callback)
nad_widget.on_select_menu(test_select_menu_callback)
nad_widget.on_move_node(test_move_node_callback)
nad_widget.on_move_text_node(test_move_text_node_callback)

#finally, display the widgets
widgets.HBox([nad_widget, out1])